In [1]:
# Import packages and read-in files

import numpy as np
import pandas as pd
import xpress as xp
from datetime import datetime, timedelta
import os
import math 

# Read in files using the explicitly defined base path
ch_0_conversion_rates = pd.read_csv('channel_0_conversion_rates.csv')
ch_0_schedule = pd.read_csv('channel_0_schedule.csv')
ch_1_conversion_rates = pd.read_csv('channel_1_conversion_rates.csv')
ch_1_schedule = pd.read_csv('channel_1_schedule.csv')
ch_2_conversion_rates = pd.read_csv('channel_2_conversion_rates.csv')
ch_2_schedule = pd.read_csv('channel_2_schedule.csv')
ch_A_schedule = pd.read_csv('channel_A_schedule.csv')
movies_df = pd.read_csv('movie_database.csv')

In [2]:
# To use right xpress and get rid of unnecessary error codes
xp.init('C:/xpressmp/bin/xpauth.xpr')
pd.options.mode.copy_on_write = True

In [3]:
# FORMATING
# Convert 'Date-Time' columns to datetime format
date_cols = ['Date']

for df in [ch_0_conversion_rates, ch_0_schedule, ch_1_conversion_rates, ch_1_schedule,
           ch_2_conversion_rates, ch_2_schedule, ch_A_schedule]:
    df['Date'] = pd.to_datetime(df['Unnamed: 0'])
    df.set_index('Date', inplace=True)
    df.drop('Unnamed: 0', axis=1, inplace = True)
   

# Convert 'Release Date' in movie_database to datetime
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'])

# Fill missing values if necessary
movies_df.fillna(0, inplace=True)

In [4]:
# Ch A 30 min compressing
from datetime import time

# Resample to 30-minute intervals (use mean of groups)
ch_A_schedule_30min = ch_A_schedule.resample('30T').mean().reset_index()

# Define start and end times for filtering
daily_start_time = time(7, 0)  # 7:00 AM
daily_end_time = time(23, 55)  # 11:55 PM

# Filter rows to include only time slots within the desired range 
ch_A_schedule_30min['time'] = ch_A_schedule_30min['Date'].dt.time
ch_A_schedule_30min = ch_A_schedule_30min[
    (ch_A_schedule_30min['time'] >= daily_start_time) &
    (ch_A_schedule_30min['time'] <= daily_end_time)
].drop(columns=['time'])
ch_A_schedule_30min.reset_index(inplace = True)
ch_A_schedule_30min.drop('index', axis = 1,inplace = True)

C:\Users\Elie\AppData\Local\Temp\ipykernel_1500\3156684494.py:5: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  ch_A_schedule_30min = ch_A_schedule.resample('30T').mean().reset_index()


In [5]:
# Ch 0 30 min compressing
from datetime import time
ch_0 = pd.DataFrame(ch_0_schedule['ad_slot_price'])


# Resample to 30-minute intervals (use mean of groups)
ch_0_schedule_30min = ch_0.resample('30T').mean().reset_index()

# Define start and end times for filtering
daily_start_time = time(7, 0)  # 7:00 AM
daily_end_time = time(23, 55)  # 11:55 PM

# Filter rows to include only time slots within the desired range 
ch_0_schedule_30min['time'] = ch_0_schedule_30min['Date'].dt.time
ch_0_schedule_30min = ch_0_schedule_30min[
    (ch_0_schedule_30min['time'] >= daily_start_time) &
    (ch_0_schedule_30min['time'] <= daily_end_time)
].drop(columns=['time'])
ch_0_schedule_30min.reset_index(inplace = True)
ch_0_schedule_30min.drop('index', axis = 1,inplace = True)

C:\Users\Elie\AppData\Local\Temp\ipykernel_1500\3603604758.py:7: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  ch_0_schedule_30min = ch_0.resample('30T').mean().reset_index()


In [6]:
# Ch 1 30 min compressing
from datetime import time
ch_1 = pd.DataFrame(ch_1_schedule['ad_slot_price'])


# Resample to 30-minute intervals (use mean of groups)
ch_1_schedule_30min = ch_1.resample('30T').mean().reset_index()

# Define start and end times for filtering
daily_start_time = time(7, 0)  # 7:00 AM
daily_end_time = time(23, 55)  # 11:55 PM

# Filter rows to include only time slots within the desired range 
ch_1_schedule_30min['time'] = ch_1_schedule_30min['Date'].dt.time
ch_1_schedule_30min = ch_1_schedule_30min[
    (ch_1_schedule_30min['time'] >= daily_start_time) &
    (ch_1_schedule_30min['time'] <= daily_end_time)
].drop(columns=['time'])
ch_1_schedule_30min.reset_index(inplace = True)
ch_1_schedule_30min.drop('index', axis = 1,inplace = True)

C:\Users\Elie\AppData\Local\Temp\ipykernel_1500\2899875544.py:7: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  ch_1_schedule_30min = ch_1.resample('30T').mean().reset_index()


In [7]:
# Ch 2 30 min compressing
from datetime import time
ch_2 = pd.DataFrame(ch_2_schedule['ad_slot_price'])


# Resample to 30-minute intervals (use mean of groups)
ch_2_schedule_30min = ch_2.resample('30T').mean().reset_index()

# Define start and end times for filtering
daily_start_time = time(7, 0)  # 7:00 AM
daily_end_time = time(23, 55)  # 11:55 PM

# Filter rows to include only time slots within the desired range 
ch_2_schedule_30min['time'] = ch_2_schedule_30min['Date'].dt.time
ch_2_schedule_30min = ch_2_schedule_30min[
    (ch_2_schedule_30min['time'] >= daily_start_time) &
    (ch_2_schedule_30min['time'] <= daily_end_time)
].drop(columns=['time'])

ch_2_schedule_30min.reset_index(inplace = True)
ch_2_schedule_30min.drop('index', axis = 1,inplace = True)

C:\Users\Elie\AppData\Local\Temp\ipykernel_1500\1338967550.py:7: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  ch_2_schedule_30min = ch_2.resample('30T').mean().reset_index()


In [8]:
all_slots = pd.DataFrame()
all_slots['0'] = ch_0_schedule_30min['ad_slot_price']
all_slots['1'] = ch_1_schedule_30min['ad_slot_price']
all_slots['2'] = ch_2_schedule_30min['ad_slot_price']
# all_slots['avg'] = ch_A_schedule_30min['ad_cost_we_charge']
all_slots['our_sell_price'] = all_slots.mean(axis = 1)
all_slots['max'] = all_slots.max(axis=1)
all_slots['min'] = all_slots.min(axis=1)
all_slots['max ch #'] = all_slots[['0','1','2']].idxmax(axis=1)
all_slots['min ch #'] = all_slots[['0','1','2']].idxmin(axis=1)

all_slots['smallest_profit'] = all_slots['our_sell_price'] - (all_slots['max'])
all_slots['greatest_profit'] = all_slots['our_sell_price'] - (all_slots['min'])


all_slots.head()

C:\Users\Elie\AppData\Local\Temp\ipykernel_1500\469590517.py:9: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  all_slots['max ch #'] = all_slots[['0','1','2']].idxmax(axis=1)
C:\Users\Elie\AppData\Local\Temp\ipykernel_1500\469590517.py:10: FutureWarning: The behavior of DataFrame.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  all_slots['min ch #'] = all_slots[['0','1','2']].idxmin(axis=1)


,0,1,2,our_sell_price,max,min,max ch #,min ch #,smallest_profit,greatest_profit
0,56995.17,25458.66,365283.35,149245.726667,365283.35,25458.66,2,1,-216037.623333,123787.066667
1,56995.17,25458.66,365283.35,149245.726667,365283.35,25458.66,2,1,-216037.623333,123787.066667
2,25479.06,16629.10,365283.35,135797.170000,365283.35,16629.10,2,1,-229486.180000,119168.070000
3,25479.06,16629.10,365283.35,135797.170000,365283.35,16629.10,2,1,-229486.180000,119168.070000
4,25479.06,16629.10,17700.00,19936.053333,25479.06,16629.10,0,1,-5543.006667,3306.953333


In [9]:
# get specifically prime time ad slots of our channel_A 
prime_time_df= pd.DataFrame(data = ch_A_schedule_30min.iloc[22:30,:])
for k in range(84):
    rest = ch_A_schedule_30min.iloc[22+(34*k):30+(34*k),:]
    prime_time_df = pd.concat([prime_time_df,rest])
print(prime_time_df)

                    Date  children_baseline_view_count  \
22   2024-10-01 18:00:00                      0.026619   
23   2024-10-01 18:30:00                      0.022203   
24   2024-10-01 19:00:00                      0.017404   
25   2024-10-01 19:30:00                      0.012819   
26   2024-10-01 20:00:00                      0.008873   
...                  ...                           ...   
2847 2024-12-23 19:30:00                      0.012819   
2848 2024-12-23 20:00:00                      0.008873   
2849 2024-12-23 20:30:00                      0.005771   
2850 2024-12-23 21:00:00                      0.003528   
2851 2024-12-23 21:30:00                      0.002026   

      adults_baseline_view_count  retirees_baseline_view_count  \
22                      0.118745                      0.034331   
23                      0.119810                      0.031795   
24                      0.119892                      0.028991   
25                      0.118990       

In [10]:
# get ad info for prime time slots
prime_ad_slots= pd.DataFrame(data = all_slots.iloc[22:30,:])
for k in range(84):
    rem = all_slots.iloc[22+(34*k):30+(34*k),:]
    prime_ad_slots = pd.concat([prime_ad_slots,rem])
print(prime_ad_slots)

              0          1          2  our_sell_price        max        min  \
22    265176.05  645960.20   69680.57   326938.940000  645960.20   69680.57   
23    265176.05  186482.72        NaN   225829.385000  265176.05  186482.72   
24     80119.92  186482.72   46807.66   104470.100000  186482.72   46807.66   
25     80119.92  186482.72   46807.66   104470.100000  186482.72   46807.66   
26     80119.92   63464.69   46807.66    63464.090000   80119.92   46807.66   
...         ...        ...        ...             ...        ...        ...   
2847  112642.91  133638.55  220948.82   155743.426667  220948.82  112642.91   
2848  112642.91  133638.55   50364.70    98882.053333  133638.55   50364.70   
2849  112642.91   15957.00   50364.70    59654.870000  112642.91   15957.00   
2850   18272.40   15957.00   50364.70    28198.033333   50364.70   15957.00   
2851   18272.40   15957.00   50364.70    28198.033333   50364.70   15957.00   

     max ch # min ch #  smallest_profit  greatest_p